In [1]:
import numpy as np
import tensorflow as tf
import cv2

import random
from os import walk

import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Convolution3D, MaxPooling3D, ZeroPadding3D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import SGD

import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
print(tf.__version__)
print(keras.__version__)

1.1.0
2.0.8


In [3]:
TRAIN_POS_DIR_PATH = "data/fire_positive_prepared_train"
TRAIN_NEG_DIR_PATH = "data/fire_negative_train"

TEST_POS_DIR_PATH = "data/fire_positive_prepared_test"
TEST_NEG_DIR_PATH = "data/fire_negative_test"

In [4]:
class VideoDataGenerator:
    
    FRAME_IN_SAMPLE = 12
    NUMBER_OF_CLASSES = 2
    WIDTH = 80
    HEIGHT = 45
    
    
    def __init__(self, pos_dir, neg_dir):
        #print("__init__")
        self.pos_dir = pos_dir
        self.neg_dir = neg_dir
        self.__init_file_lists()
        
        self.__pos_file_counter = 0
        self.__neg_file_counter = 0
        
        self.__pos_capture = None
        self.__neg_capture = None
    
    def releaseResources(self):
        if not self.__pos_capture is None:
            self.__pos_capture.release()
        
        if not self.__neg_capture is None:
            self.__neg_capture.release()
        
        self.__pos_file_counter = 0
        self.__neg_file_counter = 0
    
    def __iter__(self):
        #print("__iter__")
        return self
    
    def __next__(self):
        #print("__next__")
        y_value = random.choice([0, 1])
        y = keras.utils.to_categorical(np.array([y_value]), VideoDataGenerator.NUMBER_OF_CLASSES)
        cap = self.__get_next_cupture(y_value)
        
        succesed, sample = self.__try_get_sample_from_capture(cap, y_value)
        if not succesed:
            cap = self.__get_next_cupture(y_value)
            succesed, sample = self.__try_get_sample_from_capture(cap, y_value)
        
        return sample, y
    
    def __try_get_sample_from_capture(self, cap, is_pos):
        
        stream_fps = cap.get(cv2.CAP_PROP_FPS)
        stream_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        stream_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        
        need_resize = False
        need_cut = False
        
        can_cut = (stream_width > VideoDataGenerator.WIDTH) and (stream_height > VideoDataGenerator.HEIGHT)
        
        if (is_pos == 0) and (can_cut == False):
            need_resize = True
            need_cut = False
        
        elif (is_pos == 0) and (can_cut == True):
            need_resize = False
            need_cut = True
            
            w_margin = random.randint(0, stream_width - VideoDataGenerator.WIDTH - 1)
            h_margin = random.randint(0, stream_height - VideoDataGenerator.HEIGHT - 1)
        
        elif (is_pos == 1) and (can_cut == False):
            need_resize = True
            need_cut = False
        elif(is_pos == 1) and (can_cut == True):
            need_resize = True
            need_cut = False
        else:
            raise ValueError("VideoDataGenerator.__get_sample_from_capture: not full logical system!")
        
        if(need_resize == need_cut):
            raise ValueError("VideoDataGenerator.__get_sample_from_capture: not full logical system!")
        
        tensor_shape = (1, VideoDataGenerator.FRAME_IN_SAMPLE, VideoDataGenerator.HEIGHT, VideoDataGenerator.WIDTH, 3)
        sample_tensor = np.zeros(tensor_shape)
        for frame_num in range(VideoDataGenerator.FRAME_IN_SAMPLE):
            
            ret, frame = cap.read()
            if frame is None:
                cap.release()
                return False, None
            
            frame = frame.astype(np.float32)/255.
            if need_resize:
                frame = cv2.resize(frame, (VideoDataGenerator.WIDTH, VideoDataGenerator.HEIGHT))
            if need_cut:
                frame = frame[h_margin:h_margin + VideoDataGenerator.HEIGHT, h_margin: h_margin + VideoDataGenerator.WIDTH,:]
            
            sample_tensor[0, frame_num, :, :, :] = frame
            
        return True, sample_tensor
            
            
    def __get_next_cupture(self, is_pos):
        
        cap = self.__pos_capture if is_pos == 1 else self.__neg_capture
        if is_pos == 1:
            cur_new_file_path = self.pos_files_list[self.__pos_file_counter % len(self.pos_files_list)]
        else:
            cur_new_file_path = self.neg_files_list[self.__neg_file_counter % len(self.neg_files_list)]
        
        need_increase_counter = False
        
        if cap is None:
            cap = cv2.VideoCapture(cur_new_file_path)
            need_increase_counter = True
        
        if cap.isOpened() == False:
            cap = cv2.VideoCapture(cur_new_file_path)
            need_increase_counter = True
            
        if need_increase_counter:
            if is_pos == 1:
                self.__pos_file_counter += 1
                self.__pos_capture = cap
            else:
                self.__neg_file_counter += 1
                self.__neg_capture = cap
                
        return cap
        
    def __init_file_lists(self):
        self.pos_files_list = []
        self.neg_files_list = []
        
        for (dirpath, dirnames, filenames) in walk(self.pos_dir):
            self.pos_files_list.extend(map(lambda fn: dirpath + "/" + str(fn), filenames))
            break

        for (dirpath, dirnames, filenames) in walk(self.neg_dir):
            self.neg_files_list.extend(map(lambda fn: dirpath + "/" + str(fn), filenames))
            break

In [4]:
if False:
    vdg_test = VideoDataGenerator(TEST_POS_DIR_PATH, TEST_NEG_DIR_PATH)

    i = 0
    for X, y in vdg_test:
        i += 1

        for fn in range(VideoDataGenerator.FRAME_IN_SAMPLE):
            frame = X[0, fn, :, :, :]
            cv2.imshow('frame', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        if i == 100:
            break

    vdg_test.releaseResources()
    cv2.destroyAllWindows()
    
    X.shape

In [9]:
def get_model(summary=False):
    """ 
    Return the Keras model of the network
    """
    model = Sequential()
    
    model.add(Convolution3D(filters = 32, kernel_size = (3, 9, 9), 
                            strides = (1, 3, 3), activation = 'relu', 
                            padding='valid', name='conv_1_1', 
                            input_shape=(VideoDataGenerator.FRAME_IN_SAMPLE, 45, 80, 3)))
    
    model.add(Convolution3D(filters = 32, kernel_size = (3, 5, 5), 
                            activation='relu', 
                            padding='valid', name='conv_1_2'))
    
    model.add(Convolution3D(filters = 64, kernel_size = (3, 3, 3), 
                            activation='relu', 
                            padding='valid', name='conv_1_3'))
    
    model.add(MaxPooling3D(pool_size=(2, 2, 2), 
                           padding='valid', 
                           name = "pool_1_1"))
    
    model.add(Convolution3D(filters = 128, kernel_size = (2, 3, 3), 
                            activation='relu', 
                            padding='valid', name='conv_2_1'))
    
    model.add(MaxPooling3D(pool_size=(1, 1, 3), 
                           padding='valid', 
                           name = "pool_2_1"))
    
    model.add(Flatten())
    model.add(Dense(528, activation='relu'))
    model.add(Dropout(.25))
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(.25))
    model.add(Dense(2, activation='softmax'))
    
    if summary:
        print(model.summary())
    
    return model

model = get_model(summary=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1_1 (Conv3D)            (None, 10, 13, 24, 32)    23360     
_________________________________________________________________
conv_1_2 (Conv3D)            (None, 8, 9, 20, 32)      76832     
_________________________________________________________________
conv_1_3 (Conv3D)            (None, 6, 7, 18, 64)      55360     
_________________________________________________________________
pool_1_1 (MaxPooling3D)      (None, 3, 3, 9, 64)       0         
_________________________________________________________________
conv_2_1 (Conv3D)            (None, 2, 1, 7, 128)      147584    
_________________________________________________________________
pool_2_1 (MaxPooling3D)      (None, 2, 1, 2, 128)      0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 512)               0         
__________

In [12]:
model.compile(optimizer='SGD',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

vdg_train = VideoDataGenerator(TRAIN_POS_DIR_PATH, TRAIN_NEG_DIR_PATH)
vdg_test = VideoDataGenerator(TEST_POS_DIR_PATH, TEST_NEG_DIR_PATH)

model.fit_generator(generator = vdg_train,
                    steps_per_epoch = 1000,
                    epochs = 150,
                    validation_data = vdg_test,
                    validation_steps = 400
                   )
vdg_train.releaseResources()
vdg_test.releaseResources()

model.save("model_v2.h5")

Epoch 1/150
1000/1000 [==============================] - 71s - loss: 0.4949 - acc: 0.7250 - val_loss: 0.8575 - val_acc: 0.5975
Epoch 2/150
1000/1000 [==============================] - 91s - loss: 0.6526 - acc: 0.5760 - val_loss: 0.6933 - val_acc: 0.5125
Epoch 3/150
1000/1000 [==============================] - 106s - loss: 0.6999 - acc: 0.4710 - val_loss: 0.7071 - val_acc: 0.4875
Epoch 4/150
1000/1000 [==============================] - 114s - loss: 0.6998 - acc: 0.4700 - val_loss: 0.6929 - val_acc: 0.5175
Epoch 5/150
1000/1000 [==============================] - 109s - loss: 0.6952 - acc: 0.5240 - val_loss: 0.6934 - val_acc: 0.5075
Epoch 6/150
1000/1000 [==============================] - 96s - loss: 0.6953 - acc: 0.5000 - val_loss: 0.6999 - val_acc: 0.4975
Epoch 7/150
1000/1000 [==============================] - 113s - loss: 0.6962 - acc: 0.5140 - val_loss: 0.6943 - val_acc: 0.4500
Epoch 8/150
1000/1000 [==============================] - 117s - loss: 0.6957 - acc: 0.5050 - val_loss: 0.69

1000/1000 [==============================] - 84s - loss: 0.6695 - acc: 0.5350 - val_loss: 0.6930 - val_acc: 0.5100
Epoch 65/150
1000/1000 [==============================] - 102s - loss: 0.4556 - acc: 0.6960 - val_loss: 2.3640 - val_acc: 0.5075
Epoch 66/150
1000/1000 [==============================] - 115s - loss: 0.6079 - acc: 0.6370 - val_loss: 3.1747 - val_acc: 0.5825
Epoch 67/150
1000/1000 [==============================] - 102s - loss: 0.6894 - acc: 0.5390 - val_loss: 0.7053 - val_acc: 0.4750
Epoch 68/150
1000/1000 [==============================] - 96s - loss: 0.6458 - acc: 0.5720 - val_loss: 0.8494 - val_acc: 0.4200
Epoch 69/150
1000/1000 [==============================] - 112s - loss: 6.9742 - acc: 0.5260 - val_loss: 7.4546 - val_acc: 0.5375
Epoch 70/150
1000/1000 [==============================] - 117s - loss: 7.7850 - acc: 0.5170 - val_loss: 7.8979 - val_acc: 0.5100
Epoch 71/150
1000/1000 [==============================] - 112s - loss: 7.9623 - acc: 0.5060 - val_loss: 8.0188 -

1000/1000 [==============================] - 109s - loss: 7.5433 - acc: 0.5320 - val_loss: 7.5755 - val_acc: 0.5300
Epoch 128/150
1000/1000 [==============================] - 111s - loss: 8.0913 - acc: 0.4980 - val_loss: 7.4949 - val_acc: 0.5350
Epoch 129/150
1000/1000 [==============================] - 100s - loss: 8.3008 - acc: 0.4850 - val_loss: 9.0664 - val_acc: 0.4375
Epoch 130/150
1000/1000 [==============================] - 101s - loss: 8.0268 - acc: 0.5020 - val_loss: 7.0517 - val_acc: 0.5625
Epoch 131/150
1000/1000 [==============================] - 117s - loss: 7.6400 - acc: 0.5260 - val_loss: 8.3411 - val_acc: 0.4825
Epoch 132/150
1000/1000 [==============================] - 110s - loss: 8.2041 - acc: 0.4910 - val_loss: 7.8576 - val_acc: 0.5125
Epoch 133/150
1000/1000 [==============================] - 111s - loss: 8.3008 - acc: 0.4850 - val_loss: 7.6561 - val_acc: 0.5250
Epoch 134/150
1000/1000 [==============================] - 114s - loss: 8.4136 - acc: 0.4780 - val_loss:

In [5]:
?model.fit_generator

In [ ]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/12


In [9]:
model

In [2]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

get_available_gpus()

['/gpu:0']